In [15]:
import torch
def get_device():
  device = "cpu"
  if (torch.cuda.is_available()):
    num_gpus = torch.cuda.device_count()
    device = [-1] * num_gpus
  return device
get_device()

[-1]

In [16]:
import optuna
import os
models = ["yolo12n", "yolo12s"]
data = "./dataset/data_freeze.yaml"
epochs= 3
patience= 100
min_batch_size= 8
max_batch_size= 8
batch_size_step = 16
image_size = [32, 64]
cache = False
optimizer = ["SGD", "Adam", "AdamW", "NAdam", "RAdam", "RMSProp"]
multi_scale = [True, False]
cos_lr = [True, False]
close_mosaic_min = 1
close_mosaic_max = 100
amp = True
lr0_min = 1e-5
lr0_max = 1e-1
momentum_min = 0.5
momentum_max = 0.99
weight_decay_min = 1e-6
weight_decay_max = 1e-2
warmup_epochs_min = 1
warmup_epochs_max = 100
warmup_momentum_min = 0.5
warmup_momentum_max = 0.99
warmup_bias_lr_min = 1e-5
warmup_bias_lr_max = 0.2
box_weight_min = 1e-5
box_weight_max = 10.0
cls_weight_min = 1e-5
cls_weight_max = 10.0
dfl_weight_min = 1e-5
dfl_weight_max = 10.0
dropout_min = 1e-5
dropout_max = 0.5
max_workers = max(1, (len(os.sched_getaffinity(0)) if hasattr(os, 'sched_getaffinity') else os.cpu_count()) - 1)

In [17]:
from __future__ import annotations

from collections.abc import Callable
from collections.abc import Sequence
import functools
from typing import Any
from typing import TYPE_CHECKING

import optuna
from optuna._experimental import experimental_class
from optuna._experimental import experimental_func
from optuna._imports import try_import
from optuna.trial import TrialState

if TYPE_CHECKING:
    from optuna.study.study import ObjectiveFuncType


with try_import() as _imports:
    import wandb


@experimental_class("2.9.0")
class CustomWeightsAndBiasesCallback:
    """Callback to track Optuna trials with Weights & Biases.

    This callback enables tracking of Optuna study in
    Weights & Biases. The study is tracked as a single experiment
    run, where all suggested hyperparameters and optimized metrics
    are logged and plotted as a function of optimizer steps.

    .. note::
        User needs to be logged in to Weights & Biases before
        using this callback in online mode. For more information, please
        refer to `wandb setup <https://docs.wandb.ai/quickstart#1-set-up-wandb>`_.

    .. note::
        Users who want to run multiple Optuna studies within the same process
        should call ``wandb.finish()`` between subsequent calls to
        :meth:`optuna.study.Study.optimize`. Calling ``wandb.finish()`` is not necessary
        if you are running one Optuna study per process.

    .. note::
        To ensure correct trial order in Weights & Biases, this callback
        should only be used with ``study.optimize(n_jobs=1)``.


    Example:

        Add Weights & Biases callback to Optuna optimization.

        .. code::

            import optuna
            from optuna_integration.wandb import WeightsAndBiasesCallback


            def objective(trial):
                x = trial.suggest_float("x", -10, 10)
                return (x - 2) ** 2


            study = optuna.create_study()

            wandb_kwargs = {"project": "my-project"}
            wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)

            study.optimize(objective, n_trials=10, callbacks=[wandbc])



        Weights & Biases logging in multirun mode.

        .. code::

            import optuna
            from optuna_integration.wandb import WeightsAndBiasesCallback

            wandb_kwargs = {"project": "my-project"}
            wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs, as_multirun=True)


            @wandbc.track_in_wandb()
            def objective(trial):
                x = trial.suggest_float("x", -10, 10)
                return (x - 2) ** 2


            study = optuna.create_study()
            study.optimize(objective, n_trials=10, callbacks=[wandbc])


    Args:
        metric_name:
            Name assigned to optimized metric. In case of multi-objective optimization,
            list of names can be passed. Those names will be assigned
            to metrics in the order returned by objective function.
            If single name is provided, or this argument is left to default value,
            it will be broadcasted to each objective with a number suffix in order
            returned by objective function e.g. two objectives and default metric name
            will be logged as ``value_0`` and ``value_1``. The number of metrics must be
            the same as the number of values objective function returns.
        wandb_kwargs:
            Set of arguments passed when initializing Weights & Biases run.
            Please refer to `Weights & Biases API documentation
            <https://docs.wandb.ai/ref/python/init>`_ for more details.
        as_multirun:
            Creates new runs for each trial. Useful for generating W&B Sweeps like
            panels (for ex., parameter importance, parallel coordinates, etc).

    """

    def __init__(
        self,
        metric_name: str | Sequence[str] = "value",
        wandb_kwargs: dict[str, Any] | None = None,
        as_multirun: bool = False,
    ) -> None:
        _imports.check()

        if not isinstance(metric_name, Sequence):
            raise TypeError(
                "Expected metric_name to be string or sequence of strings, got {}.".format(
                    type(metric_name)
                )
            )

        self._metric_name = metric_name
        self._wandb_kwargs = wandb_kwargs or {}
        self._as_multirun = as_multirun

        if not self._as_multirun:
            self._initialize_run()

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial) -> None:
        # Failed and pruned trials have `None` as values.
        metrics = {}
        values: list = trial.values
        run = wandb.run

        if values is not None:
            if isinstance(self._metric_name, str):
                if len(values) > 1:
                    # Broadcast default name for multi-objective optimization.
                    names = ["{}_{}".format(self._metric_name, i) for i in range(len(values))]

                else:
                    names = [self._metric_name]

            else:
                if len(self._metric_name) != len(values):
                    raise ValueError(
                        "Running multi-objective optimization "
                        "with {} objective values, but {} names specified. "
                        "Match objective values and names, or use default broadcasting.".format(
                            len(values), len(self._metric_name)
                        )
                    )

                else:
                    names = [*self._metric_name]

            metrics = {name: value for name, value in zip(names, values)}   

        if self._as_multirun:
            metrics["trial_number"] = trial.number

        attributes = {"direction": [d.name for d in study.directions]}

        step = trial.number if wandb.run else None

        # Might create extra runs if a user logs in wandb but doesn't use the decorator.
        
        if not run:
            run = self._initialize_run()
            run.name = f"trial/{trial.number}"

        run.log({**trial.params, **metrics}, step=step)

        if self._as_multirun:
            run.config.update({**attributes, **trial.params})  # type: ignore[no-untyped-call]
            run.tags = tuple(self._wandb_kwargs.get("tags", ())) + (study.study_name,)

            if trial.state == TrialState.COMPLETE:
                run.log({
                    "status": "completed",
                    "trial_state": "COMPLETE"
                    })
                run.finish()
                
            elif trial.state == TrialState.PRUNED:
                run.log({
                    "status": "pruned",
                    "trial_state": "PRUNED"
                })  
                run.mark_preempting()
            elif trial.state == TrialState.FAIL:
                run.log({
                    "status": "failed",
                    "trial_state": "FAIL",
                    "error": "Trial failed during execution"
                    })
                run.mark_preempting() 
            
        else:
            run.config.update(attributes)  # type: ignore[no-untyped-call]
    

    def _initialize_run(self) -> "wandb.sdk.wandb_run.Run":
        """Initializes Weights & Biases run."""
        run = wandb.init(**self._wandb_kwargs)
        if not isinstance(run, wandb.sdk.wandb_run.Run):
            raise RuntimeError(
                "Cannot create a Run. "
                "Expected wandb.sdk.wandb_run.Run as a return. "
                f"Got: {type(run)}."
            )
        return run


In [18]:
import torch
import gc
import datetime as dt
import os
from ultralytics import YOLO
import wandb

def train(trial):
    # Configurar nomes e groups
    today = dt.datetime.utcnow().strftime("%Y-%m-%d")
    group_name = f"study-{today}__build_{os.getenv('GIT_SHA','local')}__data_v42"
    run_name = f"trial/{trial.number}"
    
    # Inicializar wandb no INÍCIO da trial
    run = wandb.init(
        project="yolo-optimization-monitor",
        entity="levybessa-puc",
        name=run_name,
        group=group_name,
        tags=["YOLO", "optuna"],
        reinit=True
    )
    
    wandb.log({
        "status": "running",
        "trial_number": trial.number,
        "stage": "initializing",
    })
    
    print(f"🚀 Started wandb run: {run_name} (ID: {run.id})")
    
    try:
        
        # Model selection
        model_name = trial.suggest_categorical("model", models)
        
        # Basic training parameters
        batch_size = trial.suggest_int("batch_size", min_batch_size, max_batch_size, step=batch_size_step)
        imgsz = trial.suggest_categorical("imgsz", image_size)
        
        # Optimizer
        optimizer_name = trial.suggest_categorical("optimizer", optimizer)
        
        # Training settings
        multi_scale_enabled = trial.suggest_categorical("multi_scale", multi_scale)
        cos_lr_enabled = trial.suggest_categorical("cos_lr", cos_lr)
        close_mosaic = trial.suggest_int("close_mosaic", close_mosaic_min, close_mosaic_max)
        
        # Learning rate parameters
        lr0 = trial.suggest_float("lr0", lr0_min, lr0_max, log=True)
        momentum = trial.suggest_float("momentum", momentum_min, momentum_max)
        weight_decay = trial.suggest_float("weight_decay", weight_decay_min, weight_decay_max, log=True)
        
        # Warm-up parameters
        warmup_epochs = trial.suggest_int("warmup_epochs", warmup_epochs_min, warmup_epochs_max)
        warmup_momentum = trial.suggest_float("warmup_momentum", warmup_momentum_min, warmup_momentum_max)
        warmup_bias_lr = trial.suggest_float("warmup_bias_lr", warmup_bias_lr_min, warmup_bias_lr_max)
        
        # Loss function weights
        box_weight = trial.suggest_float("box_weight", box_weight_min, box_weight_max)
        cls_weight = trial.suggest_float("cls_weight", cls_weight_min, cls_weight_max)
        dfl_weight = trial.suggest_float("dfl_weight", dfl_weight_min, dfl_weight_max)
        
        # Dropout
        dropout = trial.suggest_float("dropout", dropout_min, dropout_max)
        
        # ===== CONFIGURAR WANDB COM HIPERPARÂMETROS =====
        hyperparams = {
            "trial_number": trial.number,
            "model": model_name,
            "batch_size": batch_size,
            "imgsz": imgsz,
            "optimizer": optimizer_name,
            "multi_scale": multi_scale_enabled,
            "cos_lr": cos_lr_enabled,
            "close_mosaic": close_mosaic,
            "lr0": lr0,
            "momentum": momentum,
            "weight_decay": weight_decay,
            "warmup_epochs": warmup_epochs,
            "warmup_momentum": warmup_momentum,
            "warmup_bias_lr": warmup_bias_lr,
            "box_weight": box_weight,
            "cls_weight": cls_weight,
            "dfl_weight": dfl_weight,
            "dropout": dropout,
            # Configurações fixas
            "epochs": epochs,
            "patience": patience,
            "cache": cache,
            "amp": amp,
            "data": data,
            "max_workers": max_workers
        }
        
        wandb.config.update(hyperparams)
        
        print(f"📋 Trial {trial.number} parameters:")
        print(f"   Model: {model_name}, Batch: {batch_size}, ImgSz: {imgsz}")
        print(f"   Optimizer: {optimizer_name}, LR: {lr0:.6f}, Momentum: {momentum:.3f}")
        print(f"   Group: {group_name}")
        print(f"   Run: {run_name}")
        print("-" * 70)
        
        
        model = YOLO(f"{model_name}.pt")
        

        print(f"🏋️ Starting training for trial {trial.number}...")
        
        # IMPORTANTE: Usar wandb ativo (project=None)
        results = model.train(
            project=None,  # ← Usar wandb run ativa
            data=data,
            epochs=epochs,
            patience=patience,
            batch=batch_size,
            imgsz=imgsz,
            cache=cache,
            optimizer=optimizer_name,
            multi_scale=multi_scale_enabled,
            cos_lr=cos_lr_enabled,
            close_mosaic=close_mosaic,
            amp=amp,
            lr0=lr0,
            momentum=momentum,
            weight_decay=weight_decay,
            warmup_epochs=warmup_epochs,
            warmup_momentum=warmup_momentum,
            warmup_bias_lr=warmup_bias_lr,
            box=box_weight,
            cls=cls_weight,
            dfl=dfl_weight,
            dropout=dropout,
            verbose=False,
            save=False,
            plots=True,  # ← Gerar plots no wandb
            device=get_device(),
            workers=max_workers
        )
        
        # ===== RESULTADOS FINAIS =====
        mean_precision, mean_recall, mAP50, mAP50_90 = tuple(results.box.mean_results())
        
        # Coletar métricas adicionais se disponíveis
        final_metrics = {
            "mAP50_95": mAP50_90,
            "mean precision": mean_precision,
            "mean recall": mean_recall,
            "mAP50": mAP50,
            "status": "completed",
            "stage": "finished",
            "trial_number": trial.number,
        }
        
            
        wandb.log(final_metrics)
        wandb.finish()
        print(f"✅ Trial {trial.number} completed successfully: mAP = {mAP50_90:.4f}")
        print(f"🔗 Run URL: {run.url}")
        
        return mAP50
        
    except torch.cuda.OutOfMemoryError as e:
        # ===== ERRO CUDA OUT OF MEMORY =====
        error_info = {
            "status": "failed",
            "error_type": "CUDA_OOM", 
            "error_message": str(e),
            "trial_number": trial.number,
            "stage": "cuda_oom_error",
        }
        
        wandb.log(error_info)
        
        # Marcar como crashed
        try:
            run.mark_preempting()
        except Exception as mark_error:
            print(f"⚠️ Could not mark as preempting: {mark_error}")
        
        print(f"💥 Trial {trial.number} - CUDA Out of Memory Error!")
        print(f"   Model: {model_name}, Batch: {batch_size}, ImgSz: {imgsz}")
        print(f"   Error: {str(e)}")
        print(f"   Device: {get_device()}")
        
        # Cleanup
        if 'model' in locals():
            del model
        torch.cuda.empty_cache()
        gc.collect()
        
        return None
        
    except Exception as e:
        # ===== ERRO GERAL =====
        error_info = {
            "status": "failed",
            "error_type": type(e).__name__,
            "error_message": str(e),
            "trial_number": trial.number,
            "stage": "general_error",
        }
        
        # Adicionar contexto se variáveis existirem
        wandb.log(error_info)
        
        # Marcar como Failed
        try:
            wandb.finish(exit_code=1) 
        except Exception as mark_error:
            print(f"⚠️ Could not mark as preempting: {mark_error}")
        
        print(f"❌ Trial {trial.number} - Error: {type(e).__name__}")
        print(f"   Details: {str(e)}")
        
        # Cleanup
        if 'model' in locals():
            del model
        torch.cuda.empty_cache()
        gc.collect()
        
        return None
        
    finally:
        # ===== CLEANUP E FINALIZAÇÃO =====
        if 'model' in locals():
            try:
                del model
            except:
                pass
        
        try:
            torch.cuda.empty_cache()
            gc.collect()
        except Exception as e:
            print(f"⚠️ Cleanup failed: {e}")
        
        print(f"🧹 Trial {trial.number} cleanup completed")

In [19]:
import optuna
from optuna.pruners import BasePruner
from optuna.trial import TrialState

class ErrorPruner(BasePruner):
    def __init__(self, max_consecutive_errors=3):
        self.max_consecutive_errors = max_consecutive_errors
        self.error_count = 0

    def prune(self, study, trial):
        if trial.state == TrialState.FAIL:
            self.error_count += 1
            print(f"Trial {trial.number} failed. Consecutive errors: {self.error_count}")
            if self.error_count >= self.max_consecutive_errors:
                print(f"Many consecutive errors ({self.error_count}). Consider reviewing configuration.")
            return True
        else:
            self.error_count = 0
            return False

In [20]:
def optimize_hyperparameters(n_trials):
    study = optuna.create_study(direction="maximize",
                                study_name="YOLO_Hyperparameter_Optimization",
                                storage="sqlite:///yolo_hyperparameter_optimization.db",
                                load_if_exists=False,
                                pruner=ErrorPruner(max_consecutive_errors=5)
                                )
    study.optimize(train, n_trials=n_trials)
    
    print("Best hyperparameters found:")
    print("=" * 50)
    for key, value in study.best_params.items():
        print(f"{key}: {value}")
    
    print(f"\nBest mAP50-95: {study.best_value:.4f}")
    
    return study

In [21]:
import os
os.environ["WANDB_SILENT"] = "true"

study = optimize_hyperparameters(n_trials=50)

[I 2025-11-04 15:13:57,929] A new study created in RDB with name: YOLO_Hyperparameter_Optimization
/tmp/ipykernel_6785/2727927038.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = dt.datetime.utcnow().strftime("%Y-%m-%d")


🚀 Started wandb run: trial/0 (ID: 477wkwnu)
📋 Trial 0 parameters:
   Model: yolo12n, Batch: 8, ImgSz: 32
   Optimizer: RMSProp, LR: 0.001340, Momentum: 0.712
   Group: study-2025-11-04__build_local__data_v42
   Run: trial/0
----------------------------------------------------------------------
🏋️ Starting training for trial 0...
New https://pypi.org/project/ultralytics/8.3.225 available 😃 Update with 'pip install -U ultralytics'
Searching for 1 idle GPUs with free memory >= 20.0% and free utilization >= 0.0%...
WARNING ⚠️ No GPUs met criteria (Free Mem >= 20.0% and Free Util >= 0.0%).
Ultralytics 8.3.224 🚀 Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=3.231973100296568, cache=False, cfg=None, classes=None, close_mosaic=42, cls=1.6071073406695384, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=./dataset/data_fre

[W 2025-11-04 15:14:06,577] Trial 0 failed with parameters: {'model': 'yolo12n', 'batch_size': 8, 'imgsz': 32, 'optimizer': 'RMSProp', 'multi_scale': True, 'cos_lr': True, 'close_mosaic': 42, 'lr0': 0.0013399306253037055, 'momentum': 0.7122163254710401, 'weight_decay': 2.755320247466084e-05, 'warmup_epochs': 13, 'warmup_momentum': 0.5099347420205819, 'warmup_bias_lr': 0.059293711990152764, 'box_weight': 3.231973100296568, 'cls_weight': 1.6071073406695384, 'dfl_weight': 9.17260208042109, 'dropout': 0.22786227000043707} because of the following error: The value None could not be cast to float..
[W 2025-11-04 15:14:06,578] Trial 0 failed with value None.


🧹 Trial 0 cleanup completed


/tmp/ipykernel_6785/2727927038.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = dt.datetime.utcnow().strftime("%Y-%m-%d")


...

🧹 Trial 48 cleanup completed


/tmp/ipykernel_6785/2727927038.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = dt.datetime.utcnow().strftime("%Y-%m-%d")


🚀 Started wandb run: trial/49 (ID: ha71m4mz)
📋 Trial 49 parameters:
   Model: yolo12s, Batch: 8, ImgSz: 32
   Optimizer: RMSProp, LR: 0.042741, Momentum: 0.984
   Group: study-2025-11-04__build_local__data_v42
   Run: trial/49
----------------------------------------------------------------------
🏋️ Starting training for trial 49...
New https://pypi.org/project/ultralytics/8.3.225 available 😃 Update with 'pip install -U ultralytics'
Searching for 1 idle GPUs with free memory >= 20.0% and free utilization >= 0.0%...
WARNING ⚠️ No GPUs met criteria (Free Mem >= 20.0% and Free Util >= 0.0%).
Ultralytics 8.3.224 🚀 Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.0917745091294835, cache=False, cfg=None, classes=None, close_mosaic=76, cls=4.4228555975915365, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=./dataset/dat

[W 2025-11-04 15:43:51,192] Trial 49 failed with parameters: {'model': 'yolo12s', 'batch_size': 8, 'imgsz': 32, 'optimizer': 'RMSProp', 'multi_scale': True, 'cos_lr': True, 'close_mosaic': 76, 'lr0': 0.04274146971707261, 'momentum': 0.9839829305893462, 'weight_decay': 0.004170716478875708, 'warmup_epochs': 30, 'warmup_momentum': 0.5410243191491593, 'warmup_bias_lr': 0.06159857367193554, 'box_weight': 7.0917745091294835, 'cls_weight': 4.4228555975915365, 'dfl_weight': 3.7768060397906, 'dropout': 0.12758755665843768} because of the following error: The value None could not be cast to float..
[W 2025-11-04 15:43:51,193] Trial 49 failed with value None.


🧹 Trial 49 cleanup completed
Best hyperparameters found:
model: yolo12s
batch_size: 8
imgsz: 64
optimizer: NAdam
multi_scale: True
cos_lr: False
close_mosaic: 88
lr0: 0.005099468118027475
momentum: 0.8042817195870964
weight_decay: 0.004516680188251165
warmup_epochs: 39
warmup_momentum: 0.5346587217504501
warmup_bias_lr: 0.06155922589946828
box_weight: 3.50482502320296
cls_weight: 7.141552373294182
dfl_weight: 2.6957419151786177
dropout: 0.40411774294761516

Best mAP50-95: 0.2326


In [22]:
best_params = study.best_params
final_model = YOLO(f"{best_params['model']}.pt")

final_results = final_model.train(
    data=data,
    epochs=epochs,
    patience=patience,
    batch=best_params['batch_size'],
    imgsz=best_params['imgsz'],
    cache=cache,
    optimizer=best_params['optimizer'],
    multi_scale=best_params['multi_scale'],
    cos_lr=best_params['cos_lr'],
    close_mosaic=best_params['close_mosaic'],
    amp=amp,
    lr0=best_params['lr0'],
    momentum=best_params['momentum'],
    weight_decay=best_params['weight_decay'],
    warmup_epochs=best_params['warmup_epochs'],
    warmup_momentum=best_params['warmup_momentum'],
    warmup_bias_lr=best_params['warmup_bias_lr'],
    box=best_params['box_weight'],
    cls=best_params['cls_weight'],
    dfl=best_params['dfl_weight'],
    dropout=best_params['dropout'],
    name='best_model',
    save=True,
    plots=True
)

print("Optimization completed!")

New https://pypi.org/project/ultralytics/8.3.225 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.224 🚀 Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=3.50482502320296, cache=False, cfg=None, classes=None, close_mosaic=88, cls=7.141552373294182, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./dataset/data_freeze.yaml, degrees=0.0, deterministic=True, device=0, dfl=2.6957419151786177, dnn=False, dropout=0.40411774294761516, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=64, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005099468118027475, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12s.pt, momentum=0.8042